In [2]:
import os
#os.system('pwd')
import re
import pandas as pd
from collections import defaultdict
from sympy import symbols, simplify_logic, Or
from sympy.logic.boolalg import And, Or, Not, simplify_logic, SOPform
from itertools import product



In [ ]:
# make memory files merged
# #merge the generated mif files from the platform
#directory to the generated memory
#   should be something like './hardware/RCEHDC/mnist_example/mem/normal'
def groupMifFiles(directory):
    files = [f for f in os.listdir(directory) if f.endswith(".mif")]

    grouped_files = {}
    for file in files:
        class_index, index = file.replace(".mif", "").split("_")
        grouped_files.setdefault(index, []).append(file)

    # print (grouped_files)

    for index, file_list in grouped_files.items():
        output_file = os.path.join(directory, f"CHVs_{index}.mif")
        with open(output_file, "w") as outfile:
            for filename in sorted(file_list, key=lambda x: int(x.split("_")[0])):
                with open(os.path.join(directory, filename), "r") as infile:
                    outfile.write(infile.read().strip() + "\n")


#creat one CHV_ALL.mif
def merge_all_mif_files(directory):
    data_map = defaultdict(dict)

    # Regular expression to extract {first_index}_{second_index}.mif pattern
    pattern = re.compile(r"(\d+)_(\d+)\.mif")

    # Read all .mif files and organize them into the data_map
    for filename in os.listdir(directory):
        if filename.endswith(".mif"):
            match = pattern.match(filename)
            if match:
                first_index, second_index = map(int, match.groups())  # Convert to integers
                file_path = os.path.join(directory, filename)

                with open(file_path, "r", encoding="utf-8") as infile:
                    content = infile.read().strip().replace(" ", "")  # Remove ALL spaces
                    data_map[first_index][second_index] = content  # Store content

    # Sort first indexes in ascending order
    sorted_first_indexes = sorted(data_map.keys())

    # Determine all second indexes and sort in descending order
    all_second_indexes = sorted(
        {sec_idx for sub_dict in data_map.values() for sec_idx in sub_dict.keys()},
        reverse=True
    )

    # Create the output file
    output_file_path = os.path.join(directory,"CHV_img.mif")
    with open(output_file_path, "w", encoding="utf-8") as outfile:
        for first_index in sorted_first_indexes:
            # Concatenate all second indexes in descending order (no spaces at all)
            merged_line = "".join(
                data_map[first_index].get(sec_idx, "") for sec_idx in all_second_indexes
            )
            outfile.write(merged_line + "\n")

def read_mif_file(file_path):
    bitvectors = []
    with open(file_path, 'r') as file:
        for line in file:
            lines = line.strip()
            if lines:  # Skip empty lines
                bitvectors.append(lines)
    return bitvectors

def write_mif_file(file_path, j , bitvector):
    # print(file_path+"_"+str(j)+".mif")
    with open(file_path+"_"+str(j)+".mif", 'w') as file:
        file.write(f"{''.join(bitvector)}")
        
def generate_flag_memory_file (bitvectors):
    # Transpose bitvectors to analyze each bit position
    num_bitstreams = len(bitvectors)        # Number of bitstreams (outermost list)
    num_rows = len(bitvectors[0])          # Number of rows in each bitstream
    num_cols = len(bitvectors[0][0])       # Number of columns in each row
    
    result = []
    # transposed_bits = zip(*bitvectors)
    # print(transposed_bits)
    for row in range(num_rows):
        flag_row = []
        for col in range(num_cols):
            # Collect elements from all bitstreams at position [row][col]
            bits = [bitvectors[stream][row][col] for stream in range(num_bitstreams)]
            # Check if all elements are the same
            flag_row.append('1' if len(set(bits)) == 1 else '0')
        # Append the flag row to the result
        result.append(''.join(flag_row)) 
        
    # flag_vector = ['0' if len(set(bits)) == 1 else '1' for bits in transposed_bits]
    return result

def generate_parity_memory_file(bitvectors, flag_vector):
    parity_vector = []
    result = []
    for i in range(len(flag_vector)):
        if flag_vector[i] == '1':
                column = [bv[i] for bv in bitvectors]
                parity = column.count('1') % 2
                parity_vector.append(str(parity))
        # print("parity is : ", parity_vector)
    result.append(''.join(parity_vector))
    # print(result)
    return result

def main():
    # File paths
    directory = './hardware/RCEHDC/mnist_example/mem/normal/'
    output_dir = './hardware/RCEHDC/mnist_example/mem/normal/'
    groupMemFiles = [f for f in os.listdir(directory) if f.startswith("CHV_") and f.endswith(".mif")]
    merge_all_mif_files(directory)
    # Read the bitvectors from CHV_img.mif
    groupMemFiles.reverse()
    print(groupMemFiles)
    for i, CHVs in enumerate(groupMemFiles):
        input_file = directory+CHVs
        print(input_file, i)
        bitvectors = read_mif_file(input_file)
        # print(bitvectors)

        flag_vector = generate_flag_memory_file(bitvectors) 
        # print(flag_vector)
        write_mif_file(output_dir+"flag_memory", i, flag_vector)
        parity_vector = generate_parity_memory_file(bitvectors, flag_vector)
        write_mif_file(output_dir+"parity_memory", i, parity_vector)
        

if __name__ == "__main__":
    main()

['CHV_img.mif', 'CHV_0.mif', 'CHV_1.mif']
./hardware/RCEHDC/mnist_example/mem/normal/CHV_img.mif 0
./hardware/RCEHDC/mnist_example/mem/normal/CHV_0.mif 1
./hardware/RCEHDC/mnist_example/mem/normal/CHV_1.mif 2
